In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from ast import literal_eval
import time

In [0]:
from geomatic.geographic_handling import read_geo_point, geo_point_is_in_geojson, compute_geodesic_distance
from geomatic.utils import extract_isochrone_value
from dku_utils import get_current_project_and_variables

In [0]:
# Read recipe inputs
locations_isochrones = dataiku.Dataset("locations_isochrones")
locations_isochrones_df = locations_isochrones.get_dataframe()
customers_prepared = dataiku.Dataset("customers_prepared")
customers_prepared_df = customers_prepared.get_dataframe()

In [0]:
locations_isochrones_denormalized_df = dataiku.Dataset("locations_isochrones_denormalized").get_dataframe()

In [0]:
project, variables = get_current_project_and_variables()

In [0]:
isochrones_range_min = variables["standard"]["isochrones_for_customers"]
n_isochrone_types = len(isochrones_range_min)

In [0]:
distribution_locations = list(locations_isochrones_df["location_identifier"])
distribution_geo_points = [read_geo_point(geo_point) for geo_point in locations_isochrones_df["geo_point"]]
n_locations = len(distribution_locations)

In [0]:
customers = list(customers_prepared_df["customer_id"])
customers_geo_points = {customer: read_geo_point(geo_point)
                        for customer, geo_point
                        in zip(customers, customers_prepared_df["geo_point"])}
n_customers = len(customers)

In [0]:
n_loop_combinations = n_isochrone_types*n_locations*n_customers

In [0]:
df_distribution_locations_customers = pd.DataFrame(columns=["location_identifier",
                                                            "location_id",
                                                            "isochrone_type",
                                                            "included_customer_id",
                                                            "distance_customer_location"])

In [0]:
start_time = time.time()
index_loop = 0
loop_percents = list(range(0, 100))
print("Checking the customer locations inclusion in distribution locations isochrones ...")
for index_isochrone, isochrone_type in enumerate(isochrones_range_min):
    print("Looking for isochrones of type '{}' inclusions ...".format(isochrone_type))
    focus_isochrones = list(locations_isochrones_df[isochrone_type])
    location_isochrones = {location:literal_eval(location_isochrone)
                           for location, location_isochrone
                           in zip(distribution_locations, focus_isochrones)}

    for index_location, location in enumerate(distribution_locations):
        location_isochrone = location_isochrones[location]

        for index_customer, customer in enumerate(customers):
            try:
                customer_geo_point = customers_geo_points[customer]
                customer_is_in_isochrone = geo_point_is_in_geojson(customer_geo_point, location_isochrone)

                index_loop+= 1
                spent_time = time.time() - start_time
                loop_percent = 100*float(index_loop)/n_loop_combinations
                percent_to_exceed = loop_percents[0]
                if int(loop_percent) > percent_to_exceed:
                    del loop_percents[0]
                    log_message = "Iteration {}/{} ({}%) | Spent time : {} sec".format(index_loop,
                                                                                        n_loop_combinations,
                                                                                        loop_percent,
                                                                                        spent_time)
                    print(log_message)

            except Exception as e:
                log_message = "Exception : {}".format(str(e))
                log_message = "Isochrone type '{}' ({}/{}) | ".format(isochrone_type, index_isochrone+1, n_isochrone_types)
                log_message+= "location '{}' ({}/{}) | ".format(location, index_location+1, n_locations)
                log_message+= "customer '{}' ({}/{})".format(customer, index_customer+1, n_customers)
                print(log_message)

            if customer_is_in_isochrone:
                location_geo_point = distribution_geo_points[index_location]
                distance_customer_location = compute_geodesic_distance(customer_geo_point, location_geo_point, True)
                df_distribution_locations_customers =\
                df_distribution_locations_customers.append(
                    {"location_identifier":location,
                     "location_id":index_location,
                     "isochrone_type":isochrone_type,
                     "included_customer_id":customer,
                     "distance_customer_location":distance_customer_location},
                    ignore_index=True)

In [0]:
df_distribution_locations_customers["isochrone_value"] =\
df_distribution_locations_customers["isochrone_type"].apply(lambda x: extract_isochrone_value(x))
df_distribution_locations_customers.sort_values(by=["location_identifier", "isochrone_value"], inplace=True)
df_distribution_locations_customers.drop("isochrone_value", axis=1, inplace=True)

In [0]:
df_distribution_locations_customers["location_id"] =\
df_distribution_locations_customers["location_id"].astype(str)
locations_isochrones_denormalized_df["location_id"] =\
locations_isochrones_denormalized_df["location_id"].astype(str)

In [0]:
df_distribution_locations_customers =\
df_distribution_locations_customers.merge(locations_isochrones_denormalized_df,
                                          left_on=["location_id", "isochrone_type"],
                                          right_on=["location_id", "isochrone_type"])

In [0]:
df_distribution_locations_customers["included_customer_id"] =\
df_distribution_locations_customers["included_customer_id"].astype(str)

In [0]:
df_distribution_locations_customers["customer_id_denormalized"] =\
df_distribution_locations_customers["isochrone_id"] + "_" + df_distribution_locations_customers["included_customer_id"]

In [0]:
customers_prepared_df["customer_id"] = customers_prepared_df["customer_id"].astype(str)

In [0]:
df_distribution_locations_customers = \
df_distribution_locations_customers.merge(customers_prepared_df,
                                          left_on="included_customer_id",
                                          right_on="customer_id")
df_distribution_locations_customers.drop("customer_id", axis=1, inplace=True)

In [0]:
# Write recipe outputs
locations_customers = dataiku.Dataset("locations_customers")
locations_customers.write_with_schema(df_distribution_locations_customers)